အောက်ပါ notebook များကို chạy ရန်အတွက်၊ မလုပ်ဆောင်ထားသေးပါက၊ `text-embedding-ada-002` ကို အခြေခံမော်ဒယ်အဖြစ် အသုံးပြုသော မော်ဒယ်တစ်ခုကို တပ်ဆင်ရမည်ဖြစ်ပြီး၊ .env ဖိုင်အတွင်းတွင် deployment အမည်ကို `AZURE_OPENAI_EMBEDDINGS_ENDPOINT` ဟု သတ်မှတ်ရမည်ဖြစ်သည်။


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

နောက်တစ်ခုအနေနဲ့ ကျွန်တော်တို့က Embedding Index ကို Pandas Dataframe ထဲသို့ load လုပ်မှာဖြစ်ပါတယ်။ Embedding Index ကို `embedding_index_3m.json` ဆိုတဲ့ JSON ဖိုင်ထဲမှာ သိမ်းဆည်းထားပါတယ်။ Embedding Index မှာ ၂၀၂၃ ခုနှစ် အောက်တိုဘာလ နောက်ဆုံးအထိရှိတဲ့ YouTube transcript တစ်ခုချင်းစီအတွက် Embeddings တွေ ပါဝင်ပါတယ်။


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

နောက်တစ်ခုအနေနဲ့ `get_videos` ဆိုတဲ့ function ကိုဖန်တီးမှာဖြစ်ပြီး၊ query အတွက် Embedding Index ကိုရှာဖွေမှာဖြစ်ပါတယ်။ ဒီ function က query နဲ့အတူတူဆုံးဖြစ်တဲ့ ဗီဒီယို ၅ ခုကို return ပြန်ပေးပါလိမ့်မယ်။ function ရဲ့ လုပ်ဆောင်ပုံကတော့ အောက်ပါအတိုင်းဖြစ်ပါတယ်-

1. ပထမဦးဆုံး Embedding Index ရဲ့ copy ကိုဖန်တီးပါတယ်။
2. နောက်တစ်ခုအနေနဲ့ OpenAI Embedding API ကိုသုံးပြီး query အတွက် Embedding ကိုတွက်ချက်ပါတယ်။
3. ထို့နောက် Embedding Index ထဲမှာ `similarity` ဆိုတဲ့ column အသစ်ကိုဖန်တီးပါတယ်။ `similarity` column မှာ query Embedding နဲ့ ဗီဒီယို segment တစ်ခုချင်းစီရဲ့ Embedding ကြားက cosine similarity ပါဝင်ပါတယ်။
4. နောက်တစ်ခုအနေနဲ့ Embedding Index ကို `similarity` column အရ စစ်ထုတ်ပါတယ်။ cosine similarity 0.75 နှင့်အထက်ရှိတဲ့ ဗီဒီယိုတွေကိုသာ ထည့်သွင်းစစ်ထုတ်ပါတယ်။
5. နောက်ဆုံးမှာ Embedding Index ကို `similarity` column အရ စီစဉ်ပြီး ထိပ်ဆုံး ၅ ခုကို return ပြန်ပေးပါတယ်။


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

ဒီ function က အရမ်းရိုးရှင်းပြီး၊ ရှာဖွေမေးခွန်းရဲ့ ရလဒ်တွေကိုသာ ပုံနှိပ်ပြသပေးတာပါ။


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. ပထမဦးဆုံး၊ Embedding Index ကို Pandas Dataframe ထဲသို့ load လုပ်သည်။
2. နောက်တစ်ဆင့်မှာ အသုံးပြုသူအား query တစ်ခု ထည့်ရန် တောင်းဆိုသည်။
3. ထို့နောက် `get_videos` function ကို ခေါ်ပြီး query အတွက် Embedding Index ကို ရှာဖွေသည်။
4. နောက်ဆုံးတွင် `display_results` function ကို ခေါ်ပြီး ရလဒ်များကို အသုံးပြုသူထံ ပြသသည်။
5. အသုံးပြုသူအား နောက်ထပ် query တစ်ခု ထည့်ရန် တောင်းဆိုသည်။ ဤလုပ်ငန်းစဉ်ကို အသုံးပြုသူသည် `exit` ဟု ထည့်သည့်အထိ ဆက်လက်လုပ်ဆောင်သည်။

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc.my.png)

သင်အား query တစ်ခု ထည့်ရန် တောင်းဆိုမည်။ query တစ်ခု ထည့်ပြီး enter နှိပ်ပါ။ အဆိုပါ application သည် query နှင့် သက်ဆိုင်သော ဗီဒီယိုများစာရင်းကို ပြန်လည်ပေးမည်ဖြစ်သည်။ ထို့အပြင် မေးခွန်း၏ ဖြေကြားချက်တည်ရှိရာ ဗီဒီယိုအပိုင်းသို့ ချိတ်ဆက်ထားသော လင့်ခ်ကိုလည်း ပြန်လည်ပေးမည်ဖြစ်သည်။

စမ်းသပ်ရန် query များမှာ အောက်ပါအတိုင်းဖြစ်သည်-

- Azure Machine Learning ဆိုတာဘာလဲ?
- Convolutional neural networks များ ဘယ်လို လုပ်ဆောင်သလဲ?
- Neural network ဆိုတာဘာလဲ?
- Azure Machine Learning နှင့် Jupyter Notebooks ကို အသုံးပြုနိုင်မလား?
- ONNX ဆိုတာဘာလဲ?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**အကြောင်းကြားချက်**  
ဤစာတမ်းကို AI ဘာသာပြန်ဝန်ဆောင်မှု [Co-op Translator](https://github.com/Azure/co-op-translator) ဖြင့် ဘာသာပြန်ထားပါသည်။ ကျွန်ုပ်တို့သည် တိကျမှန်ကန်မှုအတွက် ကြိုးစားနေသော်လည်း အလိုအလျောက် ဘာသာပြန်ခြင်းတွင် အမှားများ သို့မဟုတ် မှားယွင်းချက်များ ပါဝင်နိုင်ကြောင်း သတိပြုပါရန် မေတ္တာရပ်ခံအပ်ပါသည်။ မူရင်းစာတမ်းကို မိမိဘာသာစကားဖြင့်သာ တရားဝင်အချက်အလက်အဖြစ် သတ်မှတ်သင့်ပါသည်။ အရေးကြီးသော အချက်အလက်များအတွက် လူ့ဘာသာပြန်ပညာရှင်မှ ဘာသာပြန်ခြင်းကို အကြံပြုပါသည်။ ဤဘာသာပြန်ချက်ကို အသုံးပြုမှုကြောင့် ဖြစ်ပေါ်လာနိုင်သည့် နားလည်မှုမှားယွင်းမှုများအတွက် ကျွန်ုပ်တို့ တာဝန်မယူပါ။
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
